In [2]:
import numpy as np
mean = np.array([1,2,3])
# define covariance matrix of size 3x3, with random diagonal values in the range [0,1]
cov = np.diag(np.random.rand(3))

controls = np.array([1,2])
measurements = np.array([1,2,3])
mean, cov, controls, measurements

(array([1, 2, 3]),
 array([[0.97442951, 0.        , 0.        ],
        [0.        , 0.02491636, 0.        ],
        [0.        , 0.        , 0.65967996]]),
 array([1, 2]),
 array([1, 2, 3]))

In [7]:
delta_t= 1

# prediction step
# state transition matrix A (nxn): n is the number of state variables
A = np.identity(len(mean))
# get orinetation of the robot
theta = mean[2]
# Control matrix B (nxm), where m is the number of control inputs
B = np.array([[delta_t * np.cos(theta), 0],
            [delta_t * np.sin(theta), 0],
            [0, delta_t]]
            )
# measurement noise (nxn), initialize with random values
R = np.diag(np.random.rand(len(mean)))
# update the mean
mean = np.matmul(A, mean) + np.matmul(B, controls)
# update covariance matrix
cov = np.matmul(np.matmul(A, cov), np.transpose(A)) + R

A, B, R, mean, cov

(array([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]]),
 array([[0.75390225, 0.        ],
        [0.6569866 , 0.        ],
        [0.        , 1.        ]]),
 array([[0.77259176, 0.        , 0.        ],
        [0.        , 0.64009725, 0.        ],
        [0.        , 0.        , 0.44770526]]),
 array([1.04757194, 1.83918233, 9.        ]),
 array([[1.9892287 , 0.24220743, 0.24220743],
        [0.46555392, 1.13056753, 0.46555392],
        [0.31088046, 0.31088046, 1.41826568]]))

In [8]:
# Correction step
# measurement matrix C (kxn), where k is the number of measurements
C = np.identity(len(mean))
# measurement noise (kxk), initialize with random values
Q = np.diag(np.random.rand(len(mean)))

# Kalman gain
K = np.matmul(np.matmul(cov, np.transpose(C)), np.linalg.inv(np.matmul(np.matmul(C, cov), np.transpose(C)) + Q))
# update the mean
mean = mean + np.matmul(K, measurements - np.matmul(C, mean))
# update covariance matrix
cov = np.matmul(np.identity(len(mean)) - np.matmul(K, C), cov)

C, Q, K, mean, cov

(array([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]]),
 array([[0.48645595, 0.        , 0.        ],
        [0.        , 0.14649253, 0.        ],
        [0.        , 0.        , 0.58424812]]),
 array([[0.79485629, 0.03483909, 0.01671292],
        [0.02016607, 0.874983  , 0.02662543],
        [0.02576393, 0.07090908, 0.68864118]]),
 array([0.91508429, 1.81918316, 4.87833072]),
 array([[0.38666257, 0.00510367, 0.00976449],
        [0.00980991, 0.12817847, 0.01555586],
        [0.01253302, 0.01038765, 0.40233731]]))

In [13]:
def kalaman_filter(mean, cov, controls, measurements, delta_t):
    """
    Implements the Kalman filter algorithm for state estimation.

    Args:
        mean (numpy.ndarray): The mean of the state estimate.
        cov (numpy.ndarray): The covariance matrix of the state estimate.
        controls (numpy.ndarray): The control inputs applied to the system.
        measurements (numpy.ndarray): The measurements obtained from the system.
        delta_t (float): The time step between control inputs.

    Returns:
        numpy.ndarray: The updated mean of the state estimate.
        numpy.ndarray: The updated covariance matrix of the state estimate.
    """
    
    # state transition matrix A (nxn): n is the number of state variables
    A = np.identity(len(mean))
    # get orientation of the robot
    theta = mean[2]
    # Control matrix B (nxm), where m is the number of control inputs
    B = np.array([[delta_t * np.cos(theta), 0],
                [delta_t * np.sin(theta), 0],
                [0, delta_t]]
                )
    # measurement noise (nxn), initialize with random values
    R = np.diag(np.random.rand(len(mean)))
    # measurement matrix C (kxn), where k is the number of measurements
    C = np.identity(len(mean))
    # measurement noise (kxk), initialize with random values
    Q = np.diag(np.random.rand(len(mean)))
    
    # prediction step
    # update the mean
    mean = np.matmul(A, mean) + np.matmul(B, controls)
    # update covariance matrix
    cov = np.matmul(np.matmul(A, cov), np.transpose(A)) + R
    # correction step
    # Kalman gain
    K = np.matmul(np.matmul(cov, np.transpose(C)), np.linalg.inv(np.matmul(np.matmul(C, cov), np.transpose(C)) + Q))
    # update the mean
    mean = mean + np.matmul(K, measurements - np.matmul(C, mean))
    # update covariance matrix
    cov = np.matmul(np.identity(len(mean)) - np.matmul(K, C), cov)

    return mean, cov

    
    

In [20]:
mean = np.array([1,2,3])
# define covariance matrix of size 3x3, with random diagonal values in the range [0,1]
cov = np.diag(np.random.rand(3))
controls = np.array([1,2])
measurements = np.array([1,2,3])

kalaman_filter(mean, cov, controls, measurements, delta_t)

(array([0.94855665, 2.05930592, 3.62918767]),
 array([[0.04314669, 0.        , 0.        ],
        [0.        , 0.2562228 , 0.        ],
        [0.        , 0.        , 0.25575282]]))